In [1]:
import pytest
import hls4ml
import tensorflow as tf 
import numpy as np
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Input, Dense, Activation, Conv1D, Conv2D, \
                                    Reshape, ELU, LeakyReLU, ThresholdedReLU, \
                                    PReLU, BatchNormalization, Add, Subtract, \
                                    Multiply, Average, Maximum, Minimum, Concatenate, \
                                    MaxPooling1D, MaxPooling2D, AveragePooling1D, \
                                    AveragePooling2D
import math                       
def test_dense():
    model = tf.keras.models.Sequential()
    model.add(Dense(64, 
              input_shape=(1,), 
              name='Dense', 
              use_bias=True,
              kernel_initializer='lecun_uniform',
              bias_initializer='zeros', 
              kernel_regularizer=None,
              bias_regularizer=None,
              activity_regularizer=None, 
              kernel_constraint=None, 
              bias_constraint=None))
    model.add(Activation(activation='elu', name='Activation'))
    model.compile(optimizer='adam', loss='mse')

    hls_model = hls4ml.converters.convert_from_keras_model(model)

    assert len(model.layers) + 1 == len(hls_model.get_layers())
    assert list(hls_model.get_layers())[0].attributes['class_name'] == "InputLayer"
    assert list(hls_model.get_layers())[1].attributes["class_name"] == model.layers[0]._name
    assert list(hls_model.get_layers())[2].attributes['class_name'] == model.layers[1]._name
    assert list(hls_model.get_layers())[0].attributes['input_shape'] == list(model.layers[0].input_shape[1:])
    assert list(hls_model.get_layers())[1].attributes['n_in'] == model.layers[0].input_shape[1:][0]
    assert list(hls_model.get_layers())[1].attributes['n_out'] == model.layers[0].output_shape[1:][0]
    assert list(hls_model.get_layers())[2].attributes['activation'] == str(model.layers[1].activation).split()[1]
    assert list(hls_model.get_layers())[1].attributes['activation'] == str(model.layers[0].activation).split()[1]

    hls_model.write()
    hls_model.compile()
    
    X = np.array([[1.0,]], np.float32)

    keras_predictions = model.predict(X)
    hls_predictions = hls_model.predict(X)
    print()
    print(keras_predictions)
    # compare(keras_predictions, hls_predictions, epsilon=global_eps)
    print()
    print(hls_predictions)

test_dense()

ModuleNotFoundError: No module named 'hls4ml'